In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d`


In [75]:
using GLMakie
using Colors
using GeometryBasics
using Rotations
using FileIO

include("gui_utilities.jl")

GLMakie.activate!(inline=false)

In [76]:
# Pkg.add("FileIO")

### Load Mesh

In [77]:
crazyflie_stl = load(assetpath(pwd() * "/assets/cf2_assembly.obj"));

### Create figure 

In [78]:
set_theme!(
#     font = "Arial", # inherited by layoutables if not overridden
    fontsize = 25, # inherited by layoutables if not overridden
)

f = Figure(resolution = (2560, 1920))



display(f)

GLMakie.Screen(...)

In [126]:
# clear figure 
empty!(f)

vis_params = (;aspect=1.0)
axis_params = (x_low=-2, x_high=2, y_low=-2, y_high=2, z_low=-2, z_high=2, aspect_x=1, aspect_y=1, aspect_z=1, label_size=40)
model_params = (;scale_x=10.5, scale_y=10.5, scale_z=10.5, initial_translation = Vec3f(0.0, 0.0, 0.0))
plot_params = (;count=5, titles = ["x position", "y position", "z position", "roll angle", "pitch angle"],
                            xlabels=["x pos [m]", "y pos[m]", "z pos[m]", "roll angle [°]", "pitch angle [°]"])


# Top level
g_top = f[0, 1:2] = GridLayout(alignmode=Inside())
g_left = f[1, 1] = GridLayout(alignmode=Outside(50))
g_right = f[1, 2] = GridLayout()

# Add title
supertitle1 = Label(g_top[1, 1:2], "Hover", fontsize = 60)
supertitle2 = Label(g_top[2, 1:2], "Time = 10s", fontsize = 40)

# Right Grid 
g_right_plots = g_right[1, 1]= GridLayout()
g_right_widgets = g_right[2, 1]=  GridLayout()

# Left Grid 
g_left_plots = g_left[1, 1]= GridLayout()
g_left_widgets = g_left[2, 1]=  GridLayout(tellwidth = false)


# # red box for control panel 
# Box(f[1, 2], color = (:red, 0.2), strokewidth = 0)

# # Column size adjust
colsize!(g_left, 1,  Auto(1))
colsize!(g_right, 1,  Auto(1))

# 3d axis for airplane visualization
ax1 = Axis3(g_left_plots[1, 1],
    title = "",
    limits =  (axis_params.x_low,axis_params.x_high, axis_params.y_low,axis_params.y_high, axis_params.z_low,axis_params.z_high),
    aspect = (axis_params.aspect_x, axis_params.aspect_y, axis_params.aspect_z),
    xlabel="x axis", xlabelsize=axis_params.label_size,
    ylabel="y axis", ylabelsize=axis_params.label_size,
    zlabel="z axis", zlabelsize=axis_params.label_size,
    halign =:left
)

# force 3d visualizer to have an aspect ratio of 1
rowsize!(g_left, 1, Aspect(1, vis_params.aspect))

m = mesh!(ax1, crazyflie_stl , color=:red)

scale!(m, model_params.scale_x, model_params.scale_y, model_params.scale_z)

# center model at the origin
translate!(m, model_params.initial_translation)

# orient along positive x axis
rotate!(m, Vec3f(0, 1, 0), 0) # 0.5 rad around the y axis

plots_right = Axis[]

for i in 1:plot_params.count
    plot = Axis(f, title=plot_params.titles[i], xlabel=plot_params.xlabels[i] )
    push!(plots_right, plot)

    g_right_plots[i,1] = plots_right[i]
end

# slider grid for timeline control
timeline_slider = Slider(f, range = 0:0.01:10, startvalue = 0 , linewidth = 25.0, tellheight = false,
                     halign =:left)

#timeline button
timeline_btn = Button(f, label = "Pause", tellwidth=false, halign =:center, fontsize=40)

g_left_widgets[1,1] = timeline_slider
g_left_widgets[2,1] = timeline_btn

rowsize!(g_right, 2,  Auto(0.1))

attitude_reset_btn = Button(f, label = "Reset Attitude", tellwidth=false)

g_right_widgets[1,1] = attitude_reset_btn
g_right[2, 1] = g_right_widgets


trim!(f.layout)